In [1]:
import pandas as pd
import numpy as np
import datetime
import nltk
import math

In [2]:
comments_df = pd.read_csv('../data/comments_df.csv')
no_chinese_df = pd.read_csv('../data/comments_no_chinese_df.csv')

In [3]:
print(comments_df.shape)
print(no_chinese_df.shape)

(3393, 11)
(3029, 11)


##### Change tokens columns from str type to list type

In [4]:
from ast import literal_eval

def converter(x):
    return literal_eval(x)

comments_df = pd.read_csv('../data/comments_df.csv', converters={'tokens_new':converter, 'normalized_tokens':converter})
no_chinese_df = pd.read_csv('../data/comments_no_chinese_df.csv', converters={'tokens_new':converter, 'normalized_tokens':converter})

In [5]:
comments_df.head(15)

,Unnamed: 0,id,parent_id,username,time_created,flair,body,tokens_new,normalized_tokens,normalized_tokens_count,word_count
0,0,14m8mf4,NaN,Tungsten_,2023-06-29 10:54:44,NaN,[Megathread] Supreme Court Ruling on Affirmati...,"[Megathread, Supreme, Court, Ruling, on, Affir...","[megathread, supreme, court, ruling, affirmati...",62,78
1,1,jq5du0z,t3_14m8mf4,Tungsten_,2023-06-30 11:33:11,NaN,Thanks to everyone who engaged in insightful a...,"[Thanks, to, everyone, who, engaged, in, insig...","[thank, engage, insightful, respectful, discou...",9,20
2,2,jq0dgzx,t3_14m8mf4,ProudBlackMatt,2023-06-29 11:16:15,Chinese-American,I would prefer using a process that takes into...,"[I, would, prefer, using, a, process, that, ta...","[prefer, process, take, account, poverty, inst...",52,103
3,3,jq0cg7k,t3_14m8mf4,TomatoCanned,2023-06-29 11:09:47,NaN,"u/Tungsten_, Thanks for creating a section jus...","[u/Tungsten_,, Thanks, for, creating, a, secti...","[u/tungsten_,, thank, create, section, discuss...",126,269
4,4,jq0f52k,t3_14m8mf4,bad-fengshui,2023-06-29 11:26:41,NaN,As with anything related to Asians in politics...,"[As, with, anything, related, to, Asians, in, ...","[relate, asians, politic, m, see, lot, non, as...",25,59
5,5,jq0mhrx,t3_14m8mf4,Pancake_muncher,2023-06-29 12:12:39,NaN,Yet colleges will allow alumni and doners in e...,"[Yet, colleges, will, allow, alumni, and, done...","[college, allow, alumnus, doner, easily, consi...",19,40
6,6,jq0mlbs,t3_14m8mf4,suberry,2023-06-29 12:13:16,NaN,I just hated Affirmative Action as a distracti...,"[I, just, hated, Affirmative, Action, as, a, d...","[hate, affirmative, action, distraction, banda...",78,171
7,7,jq0cvvn,t3_14m8mf4,Puzzled-Painter3301,2023-06-29 11:12:31,NaN,My own feeling is that I was never in love wit...,"[My, own, feeling, is, that, I, was, never, in...","[feeling, love, affirmative, action, possible,...",102,231
8,8,jq0jtzk,t3_14m8mf4,e9967780,2023-06-29 11:55:52,NaN,Anti Asian racism whether against East Asians ...,"[Anti, Asian, racism, whether, against, East, ...","[anti, asian, racism, east, asians, south, asi...",21,46
9,9,jq0kfzq,t3_14m8mf4,NaN,2023-06-29 11:59:41,NaN,Can we overturn legacy and athlete admissions ...,"[Can, we, overturn, legacy, and, athlete, admi...","[overturn, legacy, athlete, admission, point, ...",15,29


In [50]:
no_chinese_df.head(16)

,Unnamed: 0,id,parent_id,username,time_created,flair,body,tokens_new,normalized_tokens,normalized_tokens_count,word_count
0,0,14m8mf4,NaN,Tungsten_,2023-06-29 10:54:44,NaN,[Megathread] Supreme Court Ruling on Affirmati...,"[Megathread, Supreme, Court, Ruling, on, Affir...","[megathread, supreme, court, ruling, affirmati...",62,78
1,1,jq5du0z,t3_14m8mf4,Tungsten_,2023-06-30 11:33:11,NaN,Thanks to everyone who engaged in insightful a...,"[Thanks, to, everyone, who, engaged, in, insig...","[thank, engage, insightful, respectful, discou...",9,20
2,2,jq0dgzx,t3_14m8mf4,ProudBlackMatt,2023-06-29 11:16:15,Chinese-American,I would prefer using a process that takes into...,"[I, would, prefer, using, a, process, that, ta...","[prefer, process, take, account, poverty, inst...",52,103
3,3,jq0cg7k,t3_14m8mf4,TomatoCanned,2023-06-29 11:09:47,NaN,"u/Tungsten_, Thanks for creating a section jus...","[u/Tungsten_,, Thanks, for, creating, a, secti...","[u/tungsten_,, thank, create, section, discuss...",126,269
4,4,jq0f52k,t3_14m8mf4,bad-fengshui,2023-06-29 11:26:41,NaN,As with anything related to Asians in politics...,"[As, with, anything, related, to, Asians, in, ...","[relate, asians, politic, m, see, lot, non, as...",25,59
5,5,jq0mhrx,t3_14m8mf4,Pancake_muncher,2023-06-29 12:12:39,NaN,Yet colleges will allow alumni and doners in e...,"[Yet, colleges, will, allow, alumni, and, done...","[college, allow, alumnus, doner, easily, consi...",19,40
6,6,jq0mlbs,t3_14m8mf4,suberry,2023-06-29 12:13:16,NaN,I just hated Affirmative Action as a distracti...,"[I, just, hated, Affirmative, Action, as, a, d...","[hate, affirmative, action, distraction, banda...",78,171
7,7,jq0cvvn,t3_14m8mf4,Puzzled-Painter3301,2023-06-29 11:12:31,NaN,My own feeling is that I was never in love wit...,"[My, own, feeling, is, that, I, was, never, in...","[feeling, love, affirmative, action, possible,...",102,231
8,8,jq0jtzk,t3_14m8mf4,e9967780,2023-06-29 11:55:52,NaN,Anti Asian racism whether against East Asians ...,"[Anti, Asian, racism, whether, against, East, ...","[anti, asian, racism, east, asians, south, asi...",21,46
9,9,jq0kfzq,t3_14m8mf4,NaN,2023-06-29 11:59:41,NaN,Can we overturn legacy and athlete admissions ...,"[Can, we, overturn, legacy, and, athlete, admi...","[overturn, legacy, athlete, admission, point, ...",15,29


In [6]:
def str_to_datetime(str_):
    if isinstance(str_, str):
        return datetime.datetime.fromisoformat(str_)
    else:
        return None
        

comments_df['time_created_datetime'] = comments_df['time_created'].apply(lambda x: str_to_datetime(x))
no_chinese_df['time_created_datetime'] = no_chinese_df['time_created'].apply(lambda x: str_to_datetime(x))

## Divide comments and posts created pre-SCOTUS and post-SCOTUS ruling

#### Impute rows without dates

In [10]:
no_datetime = comments_df[pd.isna(comments_df['time_created'])]
no_datetime_no_chinese = no_chinese_df[pd.isna(no_chinese_df['time_created'])]

print(no_datetime.shape)
print(no_datetime_no_chinese.shape)

(0, 12)
(0, 12)


In [9]:
no_chinese_df

,Unnamed: 0,id,parent_id,username,time_created,flair,body,tokens_new,normalized_tokens,normalized_tokens_count,word_count,time_created_datetime
0,0,14m8mf4,NaN,Tungsten_,2023-06-29 10:54:44,NaN,[Megathread] Supreme Court Ruling on Affirmati...,"[Megathread, Supreme, Court, Ruling, on, Affir...","[megathread, supreme, court, ruling, affirmati...",62,78,2023-06-29 10:54:44
1,1,jq5du0z,t3_14m8mf4,Tungsten_,2023-06-30 11:33:11,NaN,Thanks to everyone who engaged in insightful a...,"[Thanks, to, everyone, who, engaged, in, insig...","[thank, engage, insightful, respectful, discou...",9,20,2023-06-30 11:33:11
2,2,jq0dgzx,t3_14m8mf4,ProudBlackMatt,2023-06-29 11:16:15,Chinese-American,I would prefer using a process that takes into...,"[I, would, prefer, using, a, process, that, ta...","[prefer, process, take, account, poverty, inst...",52,103,2023-06-29 11:16:15
3,3,jq0cg7k,t3_14m8mf4,TomatoCanned,2023-06-29 11:09:47,NaN,"u/Tungsten_, Thanks for creating a section jus...","[u/Tungsten_,, Thanks, for, creating, a, secti...","[u/tungsten_,, thank, create, section, discuss...",126,269,2023-06-29 11:09:47
4,4,jq0f52k,t3_14m8mf4,bad-fengshui,2023-06-29 11:26:41,NaN,As with anything related to Asians in politics...,"[As, with, anything, related, to, Asians, in, ...","[relate, asians, politic, m, see, lot, non, as...",25,59,2023-06-29 11:26:41
...,...,...,...,...,...,...,...,...,...,...,...,...
3024,3311,epcqlfa,t1_elt4c17,arientyse,2019-05-29 14:15:14,NaN,"Please don't refer to us as ""the blacks."" It's...","[Please, do, n't, refer, to, us, as, the, blac...","[refer, black, okay, black, people, harm]",6,20,2019-05-29 14:15:14
3025,3312,4ziaf0,NaN,unkle,2016-08-25 07:44:54,senyorito,From Self-Interest to Collective Morality: How...,"[From, Self, Interest, to, Collective, Moralit...","[self, interest, collective, morality, reframe...",11,20,2016-08-25 07:44:54
3026,3313,d6witvi,t3_4ziaf0,Swordbow,2016-08-25 14:47:22,NaN,The writer takes good nuggets but integrates t...,"[The, writer, takes, good, nuggets, but, integ...","[writer, take, good, nugget, integrate, questi...",156,354,2016-08-25 14:47:22
3027,3314,d6z9lhb,t3_4ziaf0,IRVCath,2016-08-27 17:02:04,"Fil-Am, 1.5 Generation","Interestingly enough, even though Filipinos ar...","[Interestingly, enough, even, though, Filipino...","[interestingly, filipino, relatively, disadvan...",51,101,2016-08-27 17:02:04


In [11]:
ruling_date = datetime.datetime(2023, 6, 29)
print(ruling_date)

2023-06-29 00:00:00


In [12]:
first_row = comments_df.iloc[0]
first_time_str = first_row['time_created']

print(type(first_time_str))

first_datetime = datetime.datetime.fromisoformat(first_time_str)
print(type(first_datetime))

print(isinstance(first_time_str, str))

<class 'str'>
<class 'datetime.datetime'>
True


In [13]:
comments_df.head(20)

,Unnamed: 0,id,parent_id,username,time_created,flair,body,tokens_new,normalized_tokens,normalized_tokens_count,word_count,time_created_datetime
0,0,14m8mf4,NaN,Tungsten_,2023-06-29 10:54:44,NaN,[Megathread] Supreme Court Ruling on Affirmati...,"[Megathread, Supreme, Court, Ruling, on, Affir...","[megathread, supreme, court, ruling, affirmati...",62,78,2023-06-29 10:54:44
1,1,jq5du0z,t3_14m8mf4,Tungsten_,2023-06-30 11:33:11,NaN,Thanks to everyone who engaged in insightful a...,"[Thanks, to, everyone, who, engaged, in, insig...","[thank, engage, insightful, respectful, discou...",9,20,2023-06-30 11:33:11
2,2,jq0dgzx,t3_14m8mf4,ProudBlackMatt,2023-06-29 11:16:15,Chinese-American,I would prefer using a process that takes into...,"[I, would, prefer, using, a, process, that, ta...","[prefer, process, take, account, poverty, inst...",52,103,2023-06-29 11:16:15
3,3,jq0cg7k,t3_14m8mf4,TomatoCanned,2023-06-29 11:09:47,NaN,"u/Tungsten_, Thanks for creating a section jus...","[u/Tungsten_,, Thanks, for, creating, a, secti...","[u/tungsten_,, thank, create, section, discuss...",126,269,2023-06-29 11:09:47
4,4,jq0f52k,t3_14m8mf4,bad-fengshui,2023-06-29 11:26:41,NaN,As with anything related to Asians in politics...,"[As, with, anything, related, to, Asians, in, ...","[relate, asians, politic, m, see, lot, non, as...",25,59,2023-06-29 11:26:41
5,5,jq0mhrx,t3_14m8mf4,Pancake_muncher,2023-06-29 12:12:39,NaN,Yet colleges will allow alumni and doners in e...,"[Yet, colleges, will, allow, alumni, and, done...","[college, allow, alumnus, doner, easily, consi...",19,40,2023-06-29 12:12:39
6,6,jq0mlbs,t3_14m8mf4,suberry,2023-06-29 12:13:16,NaN,I just hated Affirmative Action as a distracti...,"[I, just, hated, Affirmative, Action, as, a, d...","[hate, affirmative, action, distraction, banda...",78,171,2023-06-29 12:13:16
7,7,jq0cvvn,t3_14m8mf4,Puzzled-Painter3301,2023-06-29 11:12:31,NaN,My own feeling is that I was never in love wit...,"[My, own, feeling, is, that, I, was, never, in...","[feeling, love, affirmative, action, possible,...",102,231,2023-06-29 11:12:31
8,8,jq0jtzk,t3_14m8mf4,e9967780,2023-06-29 11:55:52,NaN,Anti Asian racism whether against East Asians ...,"[Anti, Asian, racism, whether, against, East, ...","[anti, asian, racism, east, asians, south, asi...",21,46,2023-06-29 11:55:52
9,9,jq0kfzq,t3_14m8mf4,NaN,2023-06-29 11:59:41,NaN,Can we overturn legacy and athlete admissions ...,"[Can, we, overturn, legacy, and, athlete, admi...","[overturn, legacy, athlete, admission, point, ...",15,29,2023-06-29 11:59:41


In [14]:
first_datetime = comments_df.iloc[0]['time_created_datetime']
print(type(first_datetime))

<class 'pandas._libs.tslibs.timestamps.Timestamp'>


In [10]:
pre_scotus = comments_df[comments_df['time_created_datetime'] < datetime.datetime(2023, 6, 29)]

In [55]:
pre_scotus

,Unnamed: 0,id,parent_id,username,time_created,flair,body,tokens_new,normalized_tokens,normalized_tokens_count,word_count,time_created_datetime
337,362,hqic5b,NaN,JesusSinfulHands,2020-07-13 11:22:51,NaN,A big gap in support for affirmative action ha...,"[A, big, gap, in, support, for, affirmative, a...","[big, gap, support, affirmative, action, emerg...",10,17,2020-07-13 11:22:51
338,363,fxy6xpv,t3_hqic5b,DJIisStupid,2020-07-13 12:38:51,NaN,wonder how this is split between income levels...,"[wonder, how, this, is, split, between, income...","[wonder, split, income, level, migration, time...",6,11,2020-07-13 12:38:51
339,364,fxy3l3x,t3_hqic5b,NaN,2020-07-13 12:12:24,NaN,Very interesting that the split for affirmativ...,"[Very, interesting, that, the, split, for, aff...","[interesting, split, affirmative, action, chin...",82,171,2020-07-13 12:12:24
340,365,fxxxtp3,t3_hqic5b,JesusSinfulHands,2020-07-13 11:26:06,NaN,Taken from this [story](http://aapidata.com/bl...,"[Taken, from, this, story](http://aapidata.com...","[take, story](http://aapidata.com, blog, asian...",81,137,2020-07-13 11:26:06
341,366,fxzzm8j,t3_hqic5b,NaN,2020-07-13 22:00:26,NaN,"Frankly as a ABC currently in college, I have ...","[Frankly, as, a, ABC, currently, in, college, ...","[frankly, abc, currently, college, recently, r...",54,98,2020-07-13 22:00:26
...,...,...,...,...,...,...,...,...,...,...,...,...
3388,3729,epcqlfa,t1_elt4c17,arientyse,2019-05-29 14:15:14,NaN,"Please don't refer to us as ""the blacks."" It's...","[Please, do, n't, refer, to, us, as, the, blac...","[refer, black, okay, black, people, harm]",6,20,2019-05-29 14:15:14
3389,3730,4ziaf0,NaN,unkle,2016-08-25 07:44:54,senyorito,From Self-Interest to Collective Morality: How...,"[From, Self, Interest, to, Collective, Moralit...","[self, interest, collective, morality, reframe...",11,20,2016-08-25 07:44:54
3390,3731,d6witvi,t3_4ziaf0,Swordbow,2016-08-25 14:47:22,NaN,The writer takes good nuggets but integrates t...,"[The, writer, takes, good, nuggets, but, integ...","[writer, take, good, nugget, integrate, questi...",156,354,2016-08-25 14:47:22
3391,3732,d6z9lhb,t3_4ziaf0,IRVCath,2016-08-27 17:02:04,"Fil-Am, 1.5 Generation","Interestingly enough, even though Filipinos ar...","[Interestingly, enough, even, though, Filipino...","[interestingly, filipino, relatively, disadvan...",51,101,2016-08-27 17:02:04


In [11]:
post_scotus = comments_df[comments_df['time_created_datetime'] >= datetime.datetime(2023, 6, 29)]

In [57]:
post_scotus

,Unnamed: 0,id,parent_id,username,time_created,flair,body,tokens_new,normalized_tokens,normalized_tokens_count,word_count,time_created_datetime
0,0,14m8mf4,NaN,Tungsten_,2023-06-29 10:54:44,NaN,[Megathread] Supreme Court Ruling on Affirmati...,"[Megathread, Supreme, Court, Ruling, on, Affir...","[megathread, supreme, court, ruling, affirmati...",62,78,2023-06-29 10:54:44
1,1,jq5du0z,t3_14m8mf4,Tungsten_,2023-06-30 11:33:11,NaN,Thanks to everyone who engaged in insightful a...,"[Thanks, to, everyone, who, engaged, in, insig...","[thank, engage, insightful, respectful, discou...",9,20,2023-06-30 11:33:11
2,2,jq0dgzx,t3_14m8mf4,ProudBlackMatt,2023-06-29 11:16:15,Chinese-American,I would prefer using a process that takes into...,"[I, would, prefer, using, a, process, that, ta...","[prefer, process, take, account, poverty, inst...",52,103,2023-06-29 11:16:15
3,3,jq0cg7k,t3_14m8mf4,TomatoCanned,2023-06-29 11:09:47,NaN,"u/Tungsten_, Thanks for creating a section jus...","[u/Tungsten_,, Thanks, for, creating, a, secti...","[u/tungsten_,, thank, create, section, discuss...",126,269,2023-06-29 11:09:47
4,4,jq0f52k,t3_14m8mf4,bad-fengshui,2023-06-29 11:26:41,NaN,As with anything related to Asians in politics...,"[As, with, anything, related, to, Asians, in, ...","[relate, asians, politic, m, see, lot, non, as...",25,59,2023-06-29 11:26:41
...,...,...,...,...,...,...,...,...,...,...,...,...
334,358,jq24mg4,t1_jq1itxi,Different-Rip-2787,2023-06-29 18:03:45,NaN,I fail to see how this is 'detrimental' to any...,"[I, fail, to, see, how, this, is, detrimental,...","[fail, detrimental, black, brown, student, qua...",37,88,2023-06-29 18:03:45
335,359,jq3gway,t1_jq3f4ro,NaN,2023-06-30 00:57:44,NaN,>We have test scores to look at\n\nIf you beli...,"[>, We, have, test, scores, to, look, at, If, ...","[test, score, look, believe, well, test, score...",45,115,2023-06-30 00:57:44
336,361,jq2ectl,t1_jq25ucy,Different-Rip-2787,2023-06-29 19:16:06,NaN,> Those same options were also available to As...,"[>, Those, same, options, were, also, availabl...","[option, available, asam, student, difference,...",30,70,2023-06-29 19:16:06
3368,3708,1belws6,NaN,DudeGuyBroPalBuddie,2024-03-14 09:02:59,NaN,How White Supremacy uses Taiwan This is a long...,"[How, White, Supremacy, uses, Taiwan, This, is...","[white, supremacy, use, taiwan, long, read, se...",1235,2644,2024-03-14 09:02:59


(OLD) Breakdown of datetimes:
- Before SCOTUS decision: 2554 rows
- After SCOTUS decision: 304 rows

- time_created is NaN: 534 rows

Since we're missing a significant number of rows with a date, collect row's date from earlier post/comment

(UPDATED) Breakdown of datetimes:
- Before SCOTUS decision: 3054 rows
- After SCOTUS decision: 339 rows

- time_created is NaN: 0 rows

#### Look at the earliest June 29 posts/comments to see if they were truly created after SCOTUS decision was annoucned

In [12]:
sorted_post_scotus = sorted(post_scotus['time_created'])

In [13]:
print(sorted_post_scotus[:20])

['2023-06-29 10:54:44', '2023-06-29 11:02:09', '2023-06-29 11:04:36', '2023-06-29 11:09:47', '2023-06-29 11:10:11', '2023-06-29 11:11:53', '2023-06-29 11:12:31', '2023-06-29 11:13:09', '2023-06-29 11:16:07', '2023-06-29 11:16:15', '2023-06-29 11:19:14', '2023-06-29 11:21:11', '2023-06-29 11:21:18', '2023-06-29 11:22:58', '2023-06-29 11:24:49', '2023-06-29 11:26:41', '2023-06-29 11:28:19', '2023-06-29 11:31:15', '2023-06-29 11:32:11', '2023-06-29 11:32:45']


Earliest June 29 comments were from after SCOTUS decision was released - meaning the date June 29 can be used as the cutoff without regarding the exact time.

## Get pairs of same-user comments pre and post SCOTUS for comparison

### Get list of usernames that commented pre and post SCOTUS decision

In [14]:
pre_scotus_usernames = pre_scotus['username'].unique()
pre_scotus_usernames = pre_scotus_usernames[~pd.isnull(pre_scotus_usernames)] # remove nan
print(len(pre_scotus_usernames))
print(pre_scotus_usernames[:10])
post_scotus_usernames = post_scotus['username'].unique()
post_scotus_usernames = post_scotus_usernames[~pd.isnull(post_scotus_usernames)] # remove nan
print(len(post_scotus_usernames))
print(post_scotus_usernames[:10])

650
['JesusSinfulHands' 'DJIisStupid' 'ohea' 'Barebacking_Bernanke' 'rubey419'
 'KillMeFastOrSlow' 'Tokidoki_Haru' 'ATF2GTalon' 'idrawfloorplan' 'unkle']
125
['Tungsten_' 'ProudBlackMatt' 'TomatoCanned' 'bad-fengshui'
 'Pancake_muncher' 'suberry' 'Puzzled-Painter3301' 'e9967780'
 'OkartoIceCream' 'ShalomHasaeyo']


In [15]:
both_usernames = np.intersect1d(pre_scotus_usernames, post_scotus_usernames)

In [16]:
print(both_usernames)
print(len(both_usernames))

['Admiral_Wen' 'AwesomeAsian' 'Different-Rip-2787' 'Gryffinclaw'
 'Jazzlike_Ad_9118' 'JerichoMassey' 'MsNewKicks' 'OkartoIceCream'
 'Puzzled-Painter3301' 'alandizzle' 'bad-monkey' 'chilispicedmango'
 'dropoutpanda' 'kevintxu' 'littleglazed' 'suberry' 'thewhizzle'
 'wildgift' 'xlsma']
19


There are 19 usernames which posted/commented pre and post SCOTUS decision.

In [17]:
matched_pre = pre_scotus[pre_scotus['username'].isin(both_usernames)]
matched_pre

,Unnamed: 0,id,parent_id,username,time_created,flair,body,tokens_new,normalized_tokens,normalized_tokens_count,word_count,time_created_datetime
351,378,fxybc65,t1_fxy6xpv,xlsma,2020-07-13 13:13:43,NaN,"Education background, immigration method (refu...","[Education, background, immigration, method, r...","[education, background, immigration, method, r...",25,41,2020-07-13 13:13:43
352,379,fy1gdmj,t1_fxy6xpv,MsNewKicks,2020-07-14 09:39:22,"First Of Her Name, Queen ABG, 나쁜 기집애, Blocker ...",Agreed. I think those two issues are at the fo...,"[Agreed, I, think, those, two, issues, are, at...","[agree, think, issue, forefront, family, gener...",27,73,2020-07-14 09:39:22
354,381,fxy7zsi,t1_fxy3l3x,Admiral_Wen,2020-07-13 12:47:18,NaN,I had thought that it's due to the influx of i...,"[I, had, thought, that, it, 's, due, to, the, ...","[think, influx, international, student, china,...",40,104,2020-07-13 12:47:18
356,385,g3buisw,t1_fxxxtp3,chilispicedmango,2020-08-30 01:32:09,PNW child of immigrants,"This is accurate, but among post-80s Mainland ...","[This, is, accurate, but, among, post-80s, Mai...","[accurate, post-80s, mainland, chinese, immigr...",18,32,2020-08-30 01:32:09
360,390,fxzrzf6,t1_fxzjhdp,kevintxu,2020-07-13 20:44:33,NaN,China have affirmative action for minority gro...,"[China, have, affirmative, action, for, minori...","[china, affirmative, action, minority, group, ...",50,91,2020-07-13 20:44:33
...,...,...,...,...,...,...,...,...,...,...,...,...
3077,3407,i3e3k4v,t3_tuqh5o,Different-Rip-2787,2022-04-04 14:26:42,NaN,As a long time Democrat I have to say that Dem...,"[As, a, long, time, Democrat, I, have, to, say...","[long, time, democrat, democrats, risk, lose, ...",19,40,2022-04-04 14:26:42
3079,3409,i403o5d,t1_i3e3k4v,wildgift,2022-04-09 00:58:42,NaN,Polls still show we support affirmative action...,"[Polls, still, show, we, support, affirmative,...","[poll, support, affirmative, action, plank, ch...",34,80,2022-04-09 00:58:42
3091,3421,cziu334,t3_43eu4h,MsNewKicks,2016-01-31 13:25:11,"First Of Her Name, Queen ABG, 나쁜 기집애, Blocker ...",I've never liked the idea of Affirmative Action.,"[I, 've, never, liked, the, idea, of, Affirmat...","[ve, like, idea, affirmative, action]",5,9,2016-01-31 13:25:11
3100,3431,dwp6ki8,t3_88rs2g,bad-monkey,2018-04-02 18:08:14,NaN,"I want to add that ""Affirmative Action"" is mor...","[I, want, to, add, that, Affirmative, Action, ...","[want, add, affirmative, action, college, admi...",30,51,2018-04-02 18:08:14


In [35]:
matched_post = post_scotus[post_scotus['username'].isin(both_usernames)]
matched_post

,Unnamed: 0,id,parent_id,username,time_created,flair,body,tokens_new,normalized_tokens,normalized_tokens_count,word_count,time_created_datetime
6,6,jq0mlbs,t3_14m8mf4,suberry,2023-06-29 12:13:16,NaN,I just hated Affirmative Action as a distracti...,"[I, just, hated, Affirmative, Action, as, a, d...","[hate, affirmative, action, distraction, banda...",78,171,2023-06-29 12:13:16
7,7,jq0cvvn,t3_14m8mf4,Puzzled-Painter3301,2023-06-29 11:12:31,NaN,My own feeling is that I was never in love wit...,"[My, own, feeling, is, that, I, was, never, in...","[feeling, love, affirmative, action, possible,...",102,231,2023-06-29 11:12:31
10,10,jq0iyws,t3_14m8mf4,OkartoIceCream,2023-06-29 11:50:34,NaN,"I want to remind people that in California, on...","[I, want, to, remind, people, that, in, Califo...","[want, remind, people, california, progressive...",104,200,2023-06-29 11:50:34
13,13,jq2fp75,t3_14m8mf4,MsNewKicks,2023-06-29 19:26:21,"First Of Her Name, Queen ABG, 나쁜 기집애, Blocker ...",There is obviously a lot to it and no real eas...,"[There, is, obviously, a, lot, to, it, and, no...","[obviously, lot, real, easy, answer, satisfy, ...",46,147,2023-06-29 19:26:21
18,19,jq0heyu,t3_14m8mf4,alandizzle,2023-06-29 11:40:55,I'm Asian. Hi.,You know what I hate?\n\nThe main subreddits w...,"[You, know, what, I, hate, The, main, subreddi...","[know, hate, main, subreddit, certain, white, ...",221,453,2023-06-29 11:40:55
...,...,...,...,...,...,...,...,...,...,...,...,...
324,347,jq0km4m,t1_jq0k0wv,littleglazed,2023-06-29 12:00:46,1.5 gen Korean Am,i'm saying that this won't solve the symptom t...,"[i, 'm, saying, that, this, wo, n't, solve, th...","[m, say, will, solve, symptom, try, solve, asi...",48,102,2023-06-29 12:00:46
325,348,jq0l02y,t1_jq0kqou,littleglazed,2023-06-29 12:03:13,1.5 gen Korean Am,"cool, and now everyone else is still fucked. b...","[cool, and, now, everyone, else, is, still, fu...","[cool, fuck, fuck, right]",4,12,2023-06-29 12:03:13
328,352,jq3f4ro,t1_jq2r9vr,Different-Rip-2787,2023-06-30 00:38:00,NaN,It's not subjective. We have test scores to lo...,"[It, 's, not, subjective, We, have, test, scor...","[subjective, test, score, look, clear, asian, ...",21,41,2023-06-30 00:38:00
334,358,jq24mg4,t1_jq1itxi,Different-Rip-2787,2023-06-29 18:03:45,NaN,I fail to see how this is 'detrimental' to any...,"[I, fail, to, see, how, this, is, detrimental,...","[fail, detrimental, black, brown, student, qua...",37,88,2023-06-29 18:03:45


## Collocations

#### Overall

In [18]:
bgs_pre = nltk.collocations.BigramCollocationFinder.from_words(pre_scotus['normalized_tokens'].sum())

In [19]:
print(pre_scotus['normalized_tokens'].sum()[:10])

['big', 'gap', 'support', 'affirmative', 'action', 'emerge', 'chinese', 'americans', 'asian', 'americans']


In [20]:
def bigramScoring(count, wordsTuple, total):
    return count
bgs_pre.nbest(bigramScoring, 50)

[('affirmative', 'action'),
 ('asian', 'americans'),
 ('asian', 'american'),
 ('white', 'people'),
 ('high', 'school'),
 ('college', 'admission'),
 ('asian', 'student'),
 ('race', 'base'),
 ('support', 'affirmative'),
 ('ivy', 'league'),
 ('legacy', 'admission'),
 ('high', 'education'),
 ('test', 'score'),
 ('chinese', 'americans'),
 ('model', 'minority'),
 ('black', 'hispanic'),
 ('asian', 'applicant'),
 ('black', 'people'),
 ('black', 'latino'),
 ('white', 'student'),
 ('student', 'body'),
 ('supreme', 'court'),
 ('asian', 'kid'),
 ('middle', 'class'),
 ('sit', 'score'),
 ('feel', 'like'),
 ('african', 'american'),
 ('african', 'americans'),
 ('admission', 'officer'),
 ('asian', 'people'),
 ('east', 'asians'),
 ('holistic', 'admission'),
 ('m', 'sure'),
 ('admission', 'process'),
 ('minority', 'group'),
 ('white', 'supremacy'),
 ('black', 'student'),
 ('white', 'applicant'),
 ('american', 'student'),
 ('low', 'income'),
 ('united', 'states'),
 ('american', 'applicant'),
 ('chinese', 

In [21]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
bgs_pre.score_ngrams(bigram_measures.likelihood_ratio)[:40]
# other options include student_t, chi_sq, likelihood_ratio, pmi

[(('affirmative', 'action'), 15345.093687770153),
 (('asian', 'americans'), 3468.0407674670573),
 (('asian', 'american'), 2028.76770905547),
 (('ivy', 'league'), 1142.063555862164),
 (('supreme', 'court'), 957.8782352707544),
 (('high', 'school'), 879.4386128419912),
 (('united', 'states'), 757.0697816034477),
 (('test', 'score'), 755.8383697810718),
 (('white', 'people'), 751.5290435540403),
 (('model', 'minority'), 687.9220195634078),
 (('college', 'admission'), 636.2422644581661),
 (('student', 'body'), 631.9969656777346),
 (('black', 'latino'), 578.9970596237888),
 (('race', 'base'), 568.236472995832),
 (('sit', 'score'), 565.0760753796471),
 (('middle', 'class'), 562.6891364129757),
 (('black', 'hispanic'), 553.3472062453639),
 (('high', 'education'), 515.0785722098726),
 (('admission', 'officer'), 486.23322026215055),
 (('legacy', 'admission'), 463.4074970974981),
 (('edward', 'blum'), 457.1116684970928),
 (('civil', 'right'), 455.2048497494735),
 (('graduation', 'rate'), 453.574

In [22]:
bgs_pre_t = bgs_pre.score_ngrams(bigram_measures.student_t)
bgs_pre_t[:40]

[(('affirmative', 'action'), 38.038682709329635),
 (('asian', 'americans'), 22.878390373673348),
 (('asian', 'american'), 18.409819535500795),
 (('white', 'people'), 13.114380292010155),
 (('high', 'school'), 12.036205753494702),
 (('college', 'admission'), 10.923337534783848),
 (('race', 'base'), 9.774192667951407),
 (('ivy', 'league'), 9.098806440748872),
 (('support', 'affirmative'), 8.952308118372523),
 (('test', 'score'), 8.89734198889062),
 (('legacy', 'admission'), 8.811041652091527),
 (('high', 'education'), 8.780762524227235),
 (('asian', 'student'), 8.67450715854658),
 (('model', 'minority'), 8.652476753334842),
 (('black', 'hispanic'), 8.549080355142959),
 (('black', 'latino'), 8.399489883803795),
 (('chinese', 'americans'), 8.366514448530655),
 (('student', 'body'), 8.351272078320035),
 (('supreme', 'court'), 8.237042547970978),
 (('sit', 'score'), 7.888807723894349),
 (('middle', 'class'), 7.887867558882465),
 (('asian', 'applicant'), 7.704547769207205),
 (('black', 'peopl

In [23]:
bgs_post = nltk.collocations.BigramCollocationFinder.from_words(post_scotus['normalized_tokens'].sum())

In [24]:
bgs_post_t = bgs_post.score_ngrams(bigram_measures.student_t)
bgs_post_t[:40]

[(('affirmative', 'action'), 10.970760096398934),
 (('asian', 'americans'), 7.919699261449686),
 (('legacy', 'admission'), 5.171609294355989),
 (('asian', 'american'), 4.618537107986249),
 (('supreme', 'court'), 4.345275267737646),
 (('black', 'student'), 4.106009461447),
 (('college', 'admission'), 3.6901859373800927),
 (('race', 'base'), 3.638878474763925),
 (('test', 'score'), 3.447129319160312),
 (('black', 'hispanic'), 3.420709030587105),
 (('white', 'people'), 3.3401004706251345),
 (('ivy', 'league'), 3.3074688153197194),
 (('white', 'student'), 3.3071859693014622),
 (('middle', 'class'), 3.2883175675367537),
 (('high', 'school'), 3.1733083477830086),
 (('student', 'body'), 3.1165039911270553),
 (('black', 'people'), 3.009441827908312),
 (('white', 'supremacy'), 2.96305356389709),
 (('western', 'country'), 2.961282159426402),
 (('base', 'affirmative'), 2.8361310276957687),
 (('united', 'states'), 2.8260412731015285),
 (('personality', 'score'), 2.8137541371315216),
 (('legacy', '

#### Chinese bi-grams

In [25]:
def find_chinese(n_gram_scores):
    chinese = []
    for i, elem in enumerate(n_gram_scores):
        if 'chinese' in elem[0]:
            chinese.append((i, elem[0], elem[1]))

    return chinese

In [26]:
chinese_pre = find_chinese(bgs_pre_t)
print(chinese_pre)

[(16, ('chinese', 'americans'), 8.366514448530655), (42, ('chinese', 'american'), 6.185391191613213), (71, ('chinese', 'immigrant'), 5.2525189145936935), (117, ('southern', 'chinese'), 4.54100838784175), (160, ('chinese', 'people'), 4.042444892344115), (295, ('mainland', 'chinese'), 3.286803556015503), (506, ('chinese', 'kid'), 2.695442841689384), (552, ('chinese', 'exclusion'), 2.619444828963571), (573, ('chinese', 'korean'), 2.594522898552078), (604, ('class', 'chinese'), 2.5396604493926693), (615, ('bear', 'chinese'), 2.5197571073176013), (631, ('chinese', 'community'), 2.4878080625319483), (703, ('han', 'chinese'), 2.4300484150692117), (752, ('chinese', 'japanese'), 2.377706378916224), (972, ('chinese', 'restaurant'), 2.201665280811699), (1013, ('chinese', 'language'), 2.1738154787308637), (1035, ('chinese', 'cantonese'), 2.162347913168167), (1054, ('chinese', 'indian'), 2.147603900301842), (1079, ('recent', 'chinese'), 2.1295834401318903), (1202, ('lot', 'chinese'), 2.012807612592

In [27]:
chinese_post = find_chinese(bgs_post_t)
print(chinese_post)

[(207, ('war', 'chinese'), 1.7066288893472845), (3149, ('break', 'chinese'), 0.9984816533108393), (3151, ('camp', 'chinese'), 0.9984816533108393), (3157, ('chinese', 'billionaire'), 0.9984816533108393), (3158, ('chinese', 'brother'), 0.9984816533108393), (3159, ('chinese', 'luxury'), 0.9984816533108393), (3160, ('chinese', 'railroad'), 0.9984816533108393), (3161, ('chinese', 'superior'), 0.9984816533108393), (3191, ('kong', 'chinese'), 0.9984816533108393), (4084, ('chinese', 'decline'), 0.9969633066216786), (4085, ('chinese', 'indians'), 0.9969633066216786), (4093, ('desire', 'chinese'), 0.9969633066216786), (4138, ('series', 'chinese'), 0.9969633066216786), (4686, ('chinese', 'city'), 0.9954449599325179), (4687, ('chinese', 'excuse'), 0.9954449599325179), (5113, ('bomb', 'chinese'), 0.9939266132433572), (5116, ('chinese', 'koreans'), 0.9939266132433572), (5117, ('chinese', 'vietnamese'), 0.9939266132433572), (5449, ('offer', 'chinese'), 0.9924082665541966), (5459, ('shit', 'chinese'),

**Notes:**
- The bi-gram 'chinese' 'americans' disappears among the list of stat sig bigrams in post-SCOTUS user-matched/not-matched corpus.
- Explanation: In pre-SCOTUS corpus, there is a post about Chinese Americans polling most negatively toward affirmative action.

#### Matched users

In [28]:
bgs_pre_matched = nltk.collocations.BigramCollocationFinder.from_words(matched_pre['normalized_tokens'].sum())

In [29]:
print(matched_pre['normalized_tokens'].sum()[:10])

['education', 'background', 'immigration', 'method', 'refugee', 'marriage', 'family', 'academic', 'opportunity', 'direct']


In [30]:
def bigramScoring(count, wordsTuple, total):
    return count
bgs_pre_matched.nbest(bigramScoring, 50)

[('affirmative', 'action'),
 ('asian', 'americans'),
 ('black', 'brown'),
 ('high', 'education'),
 ('asian', 'american'),
 ('personality', 'score'),
 ('brown', 'student'),
 ('chinese', 'americans'),
 ('support', 'affirmative'),
 ('ivy', 'league'),
 ('college', 'admission'),
 ('feel', 'like'),
 ('asian', 'student'),
 ('sit', 'score'),
 ('small', 'business'),
 ('test', 'score'),
 ('white', 'applicant'),
 ('white', 'people'),
 ('white', 'student'),
 ('asian', 'group'),
 ('harvard', 'admission'),
 ('league', 'school'),
 ('student', 'body'),
 ('aa', 'asians'),
 ('action', 'workplace'),
 ('african', 'american'),
 ('asian', 'applicant'),
 ('asian', 'kid'),
 ('chinese', 'american'),
 ('elite', 'school'),
 ('equal', 'footing'),
 ('help', 'asian'),
 ('less', 'school'),
 ('people', 'like'),
 ('register', 'voter'),
 ('safety', 'school'),
 ('aa', 'sport'),
 ('action', 'asian'),
 ('action', 'high'),
 ('action', 'want'),
 ('admission', 'officer'),
 ('american', 'latino'),
 ('american', 'voter'),
 ('b

In [31]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
bgs_pre_matched.score_ngrams(bigram_measures.likelihood_ratio)[:40]
# other options include student_t, chi_sq, likelihood_ratio, pmi

[(('affirmative', 'action'), 784.508668258475),
 (('black', 'brown'), 150.8018587855762),
 (('asian', 'americans'), 136.9263417301822),
 (('high', 'education'), 116.23834982231566),
 (('ivy', 'league'), 93.53226494574824),
 (('personality', 'score'), 91.66098888097463),
 (('brown', 'student'), 66.53532053168153),
 (('equal', 'footing'), 57.375349649505786),
 (('feel', 'like'), 53.01595608382902),
 (('asian', 'american'), 52.943213691338116),
 (('chinese', 'americans'), 52.735462445652345),
 (('small', 'business'), 52.39305599301376),
 (('student', 'body'), 47.63957102779511),
 (('register', 'voter'), 47.018795041239464),
 (('sit', 'score'), 44.75701699909371),
 (('test', 'score'), 44.75701699909371),
 (('african', 'american'), 42.36141772650757),
 (('support', 'affirmative'), 41.2521382988388),
 (('graduation', 'rate'), 40.926121291672615),
 (('white', 'applicant'), 40.49014765944735),
 (('self', 'report'), 39.901822946951505),
 (('college', 'admission'), 38.57056479099255),
 (('safety

In [32]:
bgs_pre_matched_t = bgs_pre_matched.score_ngrams(bigram_measures.student_t)

In [43]:
chinese_pre_matched = find_chinese(bgs_pre_matched_t)
print(chinese_pre_matched)

[(7, ('chinese', 'americans'), 2.920364050056883), (30, ('chinese', 'american'), 1.9051395302148162), (55, ('chinese', 'immigrant'), 1.7077095996729084), (214, ('chinese', 'ethnic'), 1.379433164583353), (219, ('bear', 'chinese'), 1.3744645363276757), (2696, ('affiliate', 'chinese'), 0.9929732985344308), (2697, ('apart', 'chinese'), 0.9929732985344308), (2699, ('chinese', 'exceptionalism'), 0.9929732985344308), (2700, ('chinese', 'object'), 0.9929732985344308), (2701, ('chinese', 'radio'), 0.9929732985344308), (2702, ('chinese', 'supermarket'), 0.9929732985344308), (2705, ('culturally', 'chinese'), 0.9929732985344308), (2708, ('exceptional', 'chinese'), 0.9929732985344308), (2711, ('fashion', 'chinese'), 0.9929732985344308), (2713, ('https://muse.jhu.edu/article/794143', 'chinese'), 0.9929732985344308), (2714, ('listen', 'chinese'), 0.9929732985344308), (2716, ('loathe', 'chinese'), 0.9929732985344308), (2717, ('mainland', 'chinese'), 0.9929732985344308), (2720, ('newspaper', 'chinese')

In [36]:
bgs_post_matched = nltk.collocations.BigramCollocationFinder.from_words(matched_post['normalized_tokens'].sum())

In [37]:
def bigramScoring(count, wordsTuple, total):
    return count
bgs_post_matched.nbest(bigramScoring, 50)

[('affirmative', 'action'),
 ('black', 'student'),
 ('asian', 'americans'),
 ('black', 'brown'),
 ('race', 'base'),
 ('student', 'body'),
 ('test', 'score'),
 ('asian', 'american'),
 ('base', 'affirmative'),
 ('community', 'college'),
 ('ivy', 'league'),
 ('school', 'student'),
 ('white', 'student'),
 ('admit', 'class'),
 ('asian', 'student'),
 ('brown', 'student'),
 ('diversity', 'campus'),
 ('diversity', 'important'),
 ('high', 'school'),
 ('immigrant', 'family'),
 ('legacy', 'admission'),
 ('low', 'income'),
 ('people', 'take'),
 ('program', 'help'),
 ('public', 'school'),
 ('public', 'university'),
 ('racial', 'diversity'),
 ('socio', 'economic'),
 ('socioeconomic', 'class'),
 ('stop', 'obsess'),
 ('student', 'actually'),
 ('student', 'admit'),
 ('student', 'population'),
 ('aa', 'legacy'),
 ('academic', 'preparation'),
 ('action', 'policy'),
 ('action', 'program'),
 ('action', 'year'),
 ('actually', 'fairly'),
 ('admission', 'policy'),
 ('admit', 'student'),
 ('asian', 'applicant'

In [38]:
bgs_post_matched.score_ngrams(bigram_measures.likelihood_ratio)[:40]
# other options include student_t, chi_sq, likelihood_ratio, pmi

[(('affirmative', 'action'), 391.9942510739762),
 (('asian', 'americans'), 94.9738695541104),
 (('black', 'student'), 60.69588237042116),
 (('test', 'score'), 57.011617367145256),
 (('ivy', 'league'), 47.396259660415424),
 (('black', 'brown'), 43.02418076007272),
 (('race', 'base'), 41.74343727284226),
 (('socio', 'economic'), 41.09373479408089),
 (('student', 'body'), 38.796853005174),
 (('stop', 'obsess'), 37.27464978431201),
 (('low', 'income'), 34.36543671928003),
 (('dynamic', 'socially'), 32.017712527602995),
 (('financial', 'aid'), 32.017712527602995),
 (('pay', 'wish'), 32.017712527602995),
 (('qualify', 'financial'), 32.017712527602995),
 (('select', 'pay'), 32.017712527602995),
 (('skin', 'color'), 32.017712527602995),
 (('socially', 'challenging'), 32.017712527602995),
 (('sociology', 'stem'), 32.017712527602995),
 (('son', 'daughter'), 32.017712527602995),
 (('immigrant', 'family'), 31.536660174008777),
 (('asian', 'american'), 28.634176332294096),
 (('vote', 'republican'),

In [39]:
bgs_post_matched_t = bgs_post_matched.score_ngrams(bigram_measures.student_t)

In [40]:
chinese_post_matched = find_chinese(bgs_post_matched_t)
print(chinese_post_matched)

[(334, ('break', 'chinese'), 0.9990925589836661), (338, ('chinese', 'koreans'), 0.9990925589836661), (552, ('chinese', 'vietnamese'), 0.9981851179673321), (746, ('poc', 'chinese'), 0.9972776769509982)]


### Next: Find differences in stat sig collocations between pre-SCOTUS and post-SCOTUS corpora